<a href="https://colab.research.google.com/github/duahauby/character-classifier-cnn-chars74k/blob/master/B%E1%BA%A3n_sao_c%E1%BB%A7a_Polarity_classification_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 13sAZl9NH8PgkInXa__mm4fxpcOGt2ttm
!gdown --id 1eB_X0DFMyKAGsxCf5_HiX4BiO9kSFwtU
!gdown --id 1Xa3HclFy9aChySDjl0vXuQtbFZFVyOTg
!gdown --id 1VSu-KbdUyApfjs2VJ-p3ccjN9zTb3zVP
!gdown --id 1utedH-ztiZchUtCEEXHzUAUWkZbLZCn7
!unzip absa_phone.zip
!unzip lm_pretrained2.zip
!rm lm_pretrained.zip
!ls

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=13sAZl9NH8PgkInXa__mm4fxpcOGt2ttm
To: /content/train_hsd.csv
100% 2.74M/2.74M [00:00<00:00, 203MB/s]
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1eB_X0DFMyKAGsxCf5_HiX4BiO9kSFwtU
To: /content/test_hsd.csv
100% 678k/678k [00:00<00:00, 131MB/s]
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import numpy as np
import torch
import random
import os
import json
import string
from unicodedata import normalize
from nltk.tokenize import word_tokenize
import re
import pandas as pd


def seed_all(seed_value):
    np.random.seed(seed_value)  # cpu vars
    torch.manual_seed(seed_value)  # cpu  vars
    random.seed(seed_value)
    os.environ["PYTHONHASHSEED"] = str(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  # gpu vars
        torch.backends.cudnn.deterministic = True  # needed
        torch.backends.cudnn.benchmark = False


def count_parameters(model) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad is True)


def clean_text(text):
    try:
      text = normalize('NFC', text)
      text = text.lower()
      text = " ".join([w for w in word_tokenize(text) if w not in string.punctuation])

      text = re.sub('\s+', ' ', text)
    except:
      text = ""
    return text



def read_data_from_csv(data_path, label2id):
    data = pd.read_csv(data_path, sep=',')
    labels =[]
    samples = []

    for index, row in data.iterrows():
      samples.append(clean_text(row['comment']))
      label = [i.strip('{').strip('}') for i in row['label'].split(';') if len(i) > 0]
      tmp = [0] * len(label2id)
      for l in label:
        tmp[label2id[l]] = 1
      labels.append(tmp)

    assert len(labels) == len(samples)
    return samples, labels


In [ ]:
import numpy as np
import torch
import itertools
from nltk.tokenize import word_tokenize


class HSDDataset(torch.utils.data.Dataset):
    def __init__(self, samples, tokenizer, max_subword_length=128):
        self.samples = samples
        self.tokenizer = tokenizer
        self.max_subword_length = max_subword_length

    def __getitem__(self, idx):
        label, text = self.samples[idx]
        sub_word_ids = self.tokenizer.encode(text)
        return sub_word_ids, label

    def __len__(self):
        return len(self.samples)

def collate_fn_padd(batch):
    text_idx, labels = zip(*batch)
    max_length = min(max([len(idx) for idx in text_idx]), 128)
    padded_input_ids = np.ones((len(text_idx), max_length), dtype=np.long)

    for i, idx in enumerate(text_idx):
        padded_input_ids[i, :len(idx)] = idx[:128]

    input_mask = np.ones(padded_input_ids.shape)
    input_mask[padded_input_ids == 1] = 0

    item = {}
    item['input_ids'] = torch.tensor(padded_input_ids)
    item['input_mask'] = torch.tensor(input_mask)

    item['labels'] = torch.tensor(labels, dtype=torch.float)

    return item

In [ ]:
import torch
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import f1_score

def train_fn(
    dataloader, model, criterion, optimizer, scheduler, device="cuda", accu_step=1
):
    model.train()
    total_loss = 0

    pbar = tqdm(dataloader, total=len(dataloader))
    for i, (batch) in enumerate(pbar):
        input_ids = batch["input_ids"].to(device)
        input_mask = batch["input_mask"].to(device)
        labels = batch["labels"].to(device)

        logits = model(input_ids, input_mask)

        # Loss calculate
        loss = criterion(logits, labels)

        # Loss backward
        loss.backward()
        if (i + 1) % accu_step == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            model.zero_grad()
            optimizer.zero_grad()

        total_loss += loss.item()

    total_loss /= len(dataloader)

    return total_loss


def validation_fn(dataloader, model, criterion, device="cuda"):
    model.eval()
    total_loss = 0
    pres, golds = [], []

    with torch.no_grad():
        pbar = tqdm(dataloader, total=len(dataloader))
        for i, (batch) in enumerate(pbar):
            input_ids = batch["input_ids"].to(device)
            input_mask = batch["input_mask"].to(device)
            labels = batch["labels"].to(device)

            logits = model(input_ids, input_mask)

            # Loss calculate
            loss = criterion(logits, labels)

            total_loss += loss.item()

            # Evaluate
            # outputs = np.argmax(logits.detach().cpu().numpy(), axis=-1)
            outputs = logits.detach().cpu().numpy() > 0.5
            labels = labels.detach().cpu().numpy()
            pres.extend(outputs)
            golds.extend(labels)

        f1 = f1_score(golds, pres, average='macro')

        print("F1 score: ", f1)

        total_loss /= len(dataloader)

        return total_loss, f1


In [ ]:
import torch
from transformers import RobertaModel
from transformers import AutoModel, AutoConfig
import torch.nn as nn
from transformers import RobertaConfig


class HSDModel(nn.Module):
    def __init__(self, model_name, num_classes, device='cuda'):
        super().__init__()
        self.model_name = model_name
        self.device = device
        self.num_classes = num_classes
        self.config = RobertaConfig.from_pretrained(
            self.model_name, from_tf=False, output_hidden_states=True
        )
        # self.config.num_hidden_layers = 12
        self.roberta = RobertaModel.from_pretrained(self.model_name, config=self.config)


        # for n, param in self.roberta.named_parameters():
        #     # if "embeddings" in n:
        #         param.requires_grad = False

        # Intent head
        self.activation = nn.Tanh()
        self.hidden_layer = nn.Linear(
            self.config.hidden_size * 1, self.config.hidden_size
        )
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Linear(
            self.config.hidden_size, self.num_classes
        )

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
    ):
        bert_outputs = self.roberta(
            input_ids,
            attention_mask,
        )

        # text_features = self.mean_pooling(bert_outputs[0], attention_mask)
        text_features = bert_outputs[0][:, 0, :]

        hidden = self.hidden_layer(text_features)
        hidden = self.activation(hidden)
        dropout = self.dropout(hidden)
        logits = torch.sigmoid(self.classifier(dropout))  # [batch, sent_len, n_labels]

        return logits

    @staticmethod
    def mean_pooling(token_embeddings, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask


In [ ]:
model_name = "./"
# model_name = "vinai/phobert-base"
training_file = "./absa_Train_phone.csv"
test_file = "./absa_Test_phone.csv"
batch_size = 32
lr = 3e-5
n_epochs = 15
seed = 96
accu_step = 1

In [ ]:
import gc
import argparse
import torch
from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.utils.data import DataLoader, SequentialSampler
from transformers import RobertaTokenizer, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
import torch.nn as nn
from sklearn.utils.class_weight import compute_class_weight


seed_all(seed_value=seed)
device = "cuda" if torch.cuda.is_available() else "cpu"

cls_labels = [
    'BATTERY',
    'CAMERA',
    'DESIGN',
    'FEATURES',
    'GENERAL',
    'OTHERS',
    'PERFORMANCE',
    'PRICE',
    'SCREEN',
    'SER&ACC',
    'STORAGE']

polarity = ["Positive", "Negative", "Neutral"]

new_labels = []
for asp in cls_labels:
  for pl in polarity:
      if asp != "OTHERS":
        new_labels.append("#".join([asp, pl]))
      else:
        new_labels.append(asp)

cls_labels = list(set(new_labels))

label2id = {cls_labels[i]: i for i in range(len(cls_labels))}
cls_labels = list(label2id.keys())
id2label = {label2id[key]: key for key in label2id}

print(cls_labels)

training_samples, training_labels = read_data_from_csv(training_file, label2id)
test_samples, test_labels = read_data_from_csv(test_file, label2id)



# Load data from file
# weight_loss = compute_class_weight(class_weight="balanced", classes=range(len(cls_labels)), y=training_labels)


_, valid_samples, _, valid_labels = train_test_split(training_samples, training_labels,
                                                                                  test_size=0.1, random_state=2024,
                                                                                  shuffle=True)


training_samples = list(zip(training_labels, training_samples))
valid_samples = list(zip(valid_labels, valid_samples))
test_samples = list(zip(test_labels, test_samples))

print('Number of training samples: ', len(training_samples))
print('Number of validation samples: ', len(valid_samples))

tokenizer = AutoTokenizer.from_pretrained(model_name)

train_dataset = HSDDataset(
    training_samples, tokenizer=tokenizer
)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn_padd,
    num_workers=1
)

valid_dataset = HSDDataset(
    valid_samples,
    tokenizer=tokenizer
)
valid_sampler = SequentialSampler(valid_dataset)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=False,
    sampler=valid_sampler,
    collate_fn=collate_fn_padd,
    num_workers=1
)

test_dataset = HSDDataset(
    test_samples,
    tokenizer=tokenizer
)
test_sampler = SequentialSampler(test_dataset)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    sampler=test_sampler,
    collate_fn=collate_fn_padd,
    num_workers=1
)

model = HSDModel(model_name, num_classes=len(cls_labels), device=device)
print('The number of parameters of the model: ', count_parameters(model))
model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ["LayerNorm.bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if (not any(nd in n for nd in no_decay))
        ],
        "weight_decay": 0.01,
    },
    {
        "params": [
            p for n, p in param_optimizer if (any(nd in n for nd in no_decay))
        ],
        "weight_decay": 0.0,
    },
]

optimizer = AdamW(optimizer_grouped_parameters, lr=lr, correct_bias=False)

total_steps = len(train_loader) * n_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=int(len(train_loader) * 0.08), num_training_steps=total_steps
)

# criterion = nn.CrossEntropyLoss(weight=torch.tensor(weight_loss, dtype=torch.float32).to(device))
# criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()

max_score = -1
for epoch in range(n_epochs):
    gc.collect()
    print("Training on epoch", epoch + 1)

    total_loss = train_fn(
        dataloader=train_loader,
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        scheduler=scheduler,
        accu_step=accu_step
    )
    print('Training loss: ', total_loss)

    total_loss, f1 = validation_fn(
        valid_loader, model, criterion, device
    )
    if max_score < f1:
        max_score = f1
        model.eval()
        torch.save( {
              "weights": model.state_dict(),
              "label2id": label2id,
              "id2label": id2label
            },
            'model.pth')
        total_loss, f1_test = validation_fn(
            test_loader, model, criterion, device
        )

        print("###############################")
    print('Validation loss', total_loss)
    print('*'*100)


['BATTERY#Positive', 'CAMERA#Neutral', 'SCREEN#Negative', 'GENERAL#Negative', 'STORAGE#Positive', 'SER&ACC#Negative', 'CAMERA#Negative', 'FEATURES#Positive', 'BATTERY#Negative', 'GENERAL#Positive', 'PRICE#Positive', 'BATTERY#Neutral', 'SER&ACC#Neutral', 'OTHERS', 'FEATURES#Negative', 'DESIGN#Positive', 'GENERAL#Neutral', 'PERFORMANCE#Negative', 'PRICE#Neutral', 'STORAGE#Negative', 'PRICE#Negative', 'SCREEN#Neutral', 'DESIGN#Neutral', 'CAMERA#Positive', 'PERFORMANCE#Positive', 'SER&ACC#Positive', 'FEATURES#Neutral', 'STORAGE#Neutral', 'SCREEN#Positive', 'PERFORMANCE#Neutral', 'DESIGN#Negative']
Number of training samples:  7786
Number of validation samples:  779


Some weights of the model checkpoint at ./ were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ./ and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The number of parameters of the model:  9934879


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training on epoch 1


...

F1 score:  0.4765835854085217


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


  0%|          | 0/70 [00:00<?, ?it/s]

<ipython-input-37-7a8d331e53fb>:24: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  padded_input_ids = np.ones((len(text_idx), max_length), dtype=np.long)
<ipython-input-37-7a8d331e53fb>:24: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use th

F1 score:  0.4187077349303583
###############################
Validation loss 0.1416596804346357
****************************************************************************************************


In [ ]:
def hsd_infer(text):
    text = clean_text(text)
    sub_word_ids = tokenizer.encode(text, max_length=model.config.max_position_embeddings-2, truncation=True, padding=False)
    mask = [1] * len(sub_word_ids)
    sub_word_ids = torch.tensor(sub_word_ids).unsqueeze(0).to(device)
    mask = torch.tensor(mask).unsqueeze(0).to(device)
    logit = model(sub_word_ids, mask)
    logit = logit.detach().cpu().numpy()[0]

    return list(zip(logit, cls_labels))

In [ ]:
hsd_infer('điện thoại gì mà vừa đắt vừa thiết kế xấu')

[(0.04414278, 'BATTERY'),
 (0.06944707, 'CAMERA'),
 (0.96735024, 'DESIGN'),
 (0.026640952, 'FEATURES'),
 (0.41111493, 'GENERAL'),
 (0.1677419, 'OTHERS'),
 (0.16732065, 'PERFORMANCE'),
 (0.79030883, 'PRICE'),
 (0.06613831, 'SCREEN'),
 (0.0794832, 'SER&ACC'),
 (0.051634025, 'STORAGE')]

In [ ]:
hsd_infer('tính năng thì ít mà pin thì lại trâu')

[(0.9855363, 'BATTERY'),
 (0.02608397, 'CAMERA'),
 (0.03988211, 'DESIGN'),
 (0.20909098, 'FEATURES'),
 (0.110634714, 'GENERAL'),
 (0.13052596, 'OTHERS'),
 (0.41936386, 'PERFORMANCE'),
 (0.0201569, 'PRICE'),
 (0.030700289, 'SCREEN'),
 (0.009840936, 'SER&ACC'),
 (0.015987806, 'STORAGE')]

In [ ]:
hsd_infer('nhà nước mà cứ thế này thì bạo động mất')

[(0.02081802, 'BATTERY'),
 (0.0090989135, 'CAMERA'),
 (0.044830892, 'DESIGN'),
 (0.05292689, 'FEATURES'),
 (0.09105211, 'GENERAL'),
 (0.26297858, 'OTHERS'),
 (0.19225553, 'PERFORMANCE'),
 (0.025589714, 'PRICE'),
 (0.031713698, 'SCREEN'),
 (0.36036184, 'SER&ACC'),
 (0.023724303, 'STORAGE')]

In [ ]:
total_loss, f1_test = validation_fn(
            test_loader, model, criterion, device
        )

<ipython-input-28-7a8d331e53fb>:24: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  padded_input_ids = np.ones((len(text_idx), max_length), dtype=np.long)


  0%|          | 0/35 [00:00<?, ?it/s]

<ipython-input-28-7a8d331e53fb>:24: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  padded_input_ids = np.ones((len(text_idx), max_length), dtype=np.long)


F1 score:  0.7243379532410835


In [ ]:
model

HSDModel(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(10000, 512, padding_idx=1)
      (position_embeddings): Embedding(130, 512, padding_idx=1)
      (token_type_embeddings): Embedding(1, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), 